In [33]:
import scanpy as sc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib
from pathlib import Path

adata = sc.read_h5ad("/Users/travishong/aml-lsc-hierarchy/data/scAML_LSPC_scran.h5ad")
adata


AnnData object with n_obs × n_vars = 4163 × 7380
    obs: 'AlignedToGenome', 'AlignedToTranscriptome', 'BMI1_expression', 'CD47_expression', 'CDK6_expression', 'CellType', 'Cluster', 'CyclingBinary', 'CyclingScore', 'FLT3_expression', 'HSC_Cycle_Primed', 'HSC_TFEB_Up', 'LSC104_DOWN', 'LSC104_UP', 'LSC17', 'MKI67_expression', 'MYC_Targets_V1', 'MutTranscripts', 'NanoporeTranscripts', 'NumberOfGenes', 'NumberOfReads', 'PCNA_expression', 'PredictionRF2', 'PredictionRefined', 'Score_B', 'Score_CTL', 'Score_GMP', 'Score_HSC', 'Score_Mono', 'Score_NK', 'Score_Plasma', 'Score_ProB', 'Score_ProMono', 'Score_Prog', 'Score_T', 'Score_cDC', 'Score_earlyEry', 'Score_lateEry', 'Score_pDC', 'TNFA_via_NFKB', 'TranscriptomeUMIs', 'WtTranscripts', 'batch', 'is_cell_control', 'log10_total_counts', 'log10_total_features_by_counts', 'pct_counts_in_top_100_features', 'pct_counts_in_top_200_features', 'pct_counts_in_top_500_features', 'pct_counts_in_top_50_features', 'total_counts', 'total_features_by_count

In [34]:
adata.obs.columns
adata.obs['CellType'].value_counts()[:20]

CellType
Prog-like    2723
HSC-like     1440
Name: count, dtype: int64

In [47]:
import os, sys

os.chdir("/Users/travishong/aml-lsc-hierarchy")  # repo root
if os.path.abspath(".") not in sys.path:
    sys.path.append(os.path.abspath("."))

print(os.getcwd())
print("src in path?", any(p.endswith("aml-lsc-hierarchy") for p in sys.path))


import scanpy as sc
import numpy as np
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import joblib

adata = sc.read_h5ad("/Users/travishong/aml-lsc-hierarchy/data/scAML_LSPC_scran.h5ad")
print(adata)
print(adata.obs["CellType"].value_counts())


/Users/travishong/aml-lsc-hierarchy
src in path? True
AnnData object with n_obs × n_vars = 4163 × 7380
    obs: 'AlignedToGenome', 'AlignedToTranscriptome', 'BMI1_expression', 'CD47_expression', 'CDK6_expression', 'CellType', 'Cluster', 'CyclingBinary', 'CyclingScore', 'FLT3_expression', 'HSC_Cycle_Primed', 'HSC_TFEB_Up', 'LSC104_DOWN', 'LSC104_UP', 'LSC17', 'MKI67_expression', 'MYC_Targets_V1', 'MutTranscripts', 'NanoporeTranscripts', 'NumberOfGenes', 'NumberOfReads', 'PCNA_expression', 'PredictionRF2', 'PredictionRefined', 'Score_B', 'Score_CTL', 'Score_GMP', 'Score_HSC', 'Score_Mono', 'Score_NK', 'Score_Plasma', 'Score_ProB', 'Score_ProMono', 'Score_Prog', 'Score_T', 'Score_cDC', 'Score_earlyEry', 'Score_lateEry', 'Score_pDC', 'TNFA_via_NFKB', 'TranscriptomeUMIs', 'WtTranscripts', 'batch', 'is_cell_control', 'log10_total_counts', 'log10_total_features_by_counts', 'pct_counts_in_top_100_features', 'pct_counts_in_top_200_features', 'pct_counts_in_top_500_features', 'pct_counts_in_top_

In [36]:
map_to_hierarchy = {
    "HSC-like": "Primitive",
    "Prog-like": "Progenitor",
}

adata.obs["hierarchy_label"] = adata.obs["CellType"].map(map_to_hierarchy)
print(adata.obs["hierarchy_label"].value_counts())


hierarchy_label
Progenitor    2723
Primitive     1440
Name: count, dtype: int64


In [37]:
adata_proc = adata.copy()

sc.pp.normalize_total(adata_proc, target_sum=1e4)
sc.pp.log1p(adata_proc)
sc.pp.highly_variable_genes(adata_proc, n_top_genes=3000, subset=True)
sc.pp.scale(adata_proc, max_value=10)

X = adata_proc.X
y = adata_proc.obs["hierarchy_label"].values
print(X.shape, np.unique(y, return_counts=True))

/Users/travishong/miniforge3/envs/aml-lsc/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)
/Users/travishong/miniforge3/envs/aml-lsc/lib/python3.11/site-packages/scanpy/preprocessing/_simple.py:391: RuntimeWarning: invalid value encountered in log1p
  np.log1p(x, out=x)


ValueError: Bin edges must be unique: Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan],
      dtype='float64').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [38]:
import numpy as np
import scanpy as sc
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import joblib

# 0. Load
adata = sc.read_h5ad("/Users/travishong/aml-lsc-hierarchy/data/scAML_LSPC_scran.h5ad")
print(adata.obs["CellType"].value_counts())

# 1. Map to hierarchy labels
map_to_hierarchy = {
    "HSC-like": "Primitive",
    "Prog-like": "Progenitor",
}
adata.obs["hierarchy_label"] = adata.obs["CellType"].map(map_to_hierarchy)
print(adata.obs["hierarchy_label"].value_counts())

# 2. Work on a copy
adata_proc = adata.copy()

# 2a. Drop cells with zero or NaN total signal
cell_sums = np.array(adata_proc.X.sum(axis=1)).flatten()
good_cells = np.isfinite(cell_sums) & (cell_sums > 0)
adata_proc = adata_proc[good_cells, :].copy()

# 2b. Drop genes that are never expressed
gene_sums = np.array(adata_proc.X.sum(axis=0)).flatten()
good_genes = np.isfinite(gene_sums) & (gene_sums > 0)
adata_proc = adata_proc[:, good_genes].copy()

print("After filtering:", adata_proc.shape)

# 3. Select HVGs on already-normalized data
sc.pp.highly_variable_genes(
    adata_proc,
    n_top_genes=3000,
    flavor="seurat",   # or "cell_ranger"
    subset=True
)


# 4. Scale
sc.pp.scale(adata_proc, max_value=10)

X = adata_proc.X
y = adata_proc.obs["hierarchy_label"].values

print("Training matrix:", X.shape, np.unique(y, return_counts=True))

# 5. Train classifier
clf = LogisticRegression(
    max_iter=2000,
    multi_class="multinomial",
    n_jobs=-1
)
clf.fit(X, y)

print("Training accuracy:", clf.score(X, y))

# 6. Save model + gene list
Path("models").mkdir(exist_ok=True)
joblib.dump(clf, "models/hierarchy_lr.joblib")
np.save("models/hierarchy_genes.npy", adata_proc.var_names.to_numpy())

print("Saved model and genes.")


CellType
Prog-like    2723
HSC-like     1440
Name: count, dtype: int64
hierarchy_label
Progenitor    2723
Primitive     1440
Name: count, dtype: int64
After filtering: (1948, 6908)
Training matrix: (1948, 3000) (array(['Primitive', 'Progenitor'], dtype=object), array([ 604, 1344]))


/Users/travishong/miniforge3/envs/aml-lsc/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1262: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


Training accuracy: 1.0
Saved model and genes.


In [40]:
from src.hierarchy_classifier import load_hierarchy_model, annotate_hierarchy

clf, ref_genes = load_hierarchy_model()
test_adata = annotate_hierarchy(adata, clf, ref_genes, label_key="hierarchy_pred")

pd.crosstab(adata.obs["hierarchy_label"], test_adata.obs["hierarchy_pred"])


hierarchy_pred,Primitive,Progenitor
hierarchy_label,,
Primitive,604,0
Progenitor,0,1344


ImportError: cannot import name 'map_zeng_celltype_to_hierarchy' from 'src.hierarchy_classifier' (/Users/travishong/aml-lsc-hierarchy/src/hierarchy_classifier.py)